In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/stroke-prediction-dataset-assignment/submission.csv
/kaggle/input/stroke-prediction-dataset-assignment/train.csv
/kaggle/input/stroke-prediction-dataset-assignment/test.csv


In [2]:
column_values = {
    'gender': ['Male', 'Female', 'Other'],
    'ever_married': ['Yes', 'No'],
    'Residence_type': ['Rural', 'Urban'],
    'smoking_status': ['formerly smoked', 'never smoked', 'smokes', 'Unknown'],
    'work_type': ['Private', 'Self-employed', 'children', 'Govt_job', 'Never_worked']
}

In [3]:
x_train = pd.read_csv('/kaggle/input/stroke-prediction-dataset-assignment/train.csv')
x_test = pd.read_csv('/kaggle/input/stroke-prediction-dataset-assignment/test.csv')

In [4]:
# columns = ['gender', 'ever_married', 'Residence_type', 'smoking_status', 'work_type', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', '']
# target_column = 'stroke'

# for column in columns:
#     # Group the data by the column and count occurrences of stroke and non-stroke cases
#     grouped = x_train.groupby([column, target_column]).size().unstack()

#     # Plotting the grouped data as a stacked bar chart
#     grouped.plot(kind='bar', stacked=True)
#     plt.xlabel(column)
#     plt.ylabel('Count')
#     plt.title(f'Stroke vs Non-Stroke Counts by {column}')
#     plt.legend(['Non-Stroke', 'Stroke'])
#     plt.show()

In [5]:
x_train.drop(['id'],axis=1,inplace=True)
# data['gender'].replace({'Female':1, 'Male':0 ,'Other':0}, inplace=True)
x_train['ever_married'].replace({'Yes':1, 'No':0 }, inplace=True)
x_train['work_type'].replace({'Private':3, 'Govt_job':2 , 'Self-employed':2 , 'children':1 , 'Never_worked':1}, inplace=True)
#data['Residence_type'].replace({'Rural':1, 'Urban':0 }, inplace=True)
x_train = pd.get_dummies(x_train,columns=["smoking_status"])
#data.drop('work_type', axis=1,inplace = True)
x_train.drop('gender', axis=1,inplace = True)
x_train.drop('Residence_type', axis=1,inplace = True)


#data_t['gender'].replace({'Female':1, 'Male':0 ,'Other':0}, inplace=True)
x_test['ever_married'].replace({'Yes':1, 'No':0 }, inplace=True)
#data_t['Residence_type'].replace({'Rural':1, 'Urban':0 }, inplace=True)
x_test = pd.get_dummies(x_test,columns=["smoking_status"])
#data_t.drop('work_type', axis=1,inplace = True)
x_test['work_type'].replace({'Private':3, 'Govt_job':2 , 'Self-employed':2 , 'children':1 , 'Never_worked':1}, inplace=True)
x_test.drop('Residence_type', axis=1,inplace = True)
x_test.drop('gender', axis=1,inplace = True)
i = x_test.drop('id',axis=1,inplace=True)
x_test.fillna(0,inplace=True)
x_train.fillna(0,inplace=True)
# data_t.info()

In [6]:
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)

In [7]:
# for column, values in column_values.items():
#     x_train[column] = x_train[column].map(lambda x: values.index(x) if x in values else x)
#     x_test[column] = x_test[column].map(lambda x: values.index(x) if x in values else x)

In [8]:
y_train = x_train['stroke']

In [9]:
x_train.drop(['stroke'], axis=1, inplace=True)

In [10]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [11]:
clf = DecisionTreeClassifier(
    criterion='gini',
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

In [12]:
clf.fit(x_train_resampled, y_train_resampled)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=10,
                       random_state=42)

In [13]:
y_pred = clf.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [14]:
temp = pd.read_csv('/kaggle/input/stroke-prediction-dataset-assignment/test.csv')
submission = pd.DataFrame({'id': temp['id'], 'stroke': y_pred})
submission.to_csv('submission.csv', index=False)